# 第9章 : カテゴリカル特徴量(変数)の取り扱い、機械学習のための新しい特徴量作成

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
df = pd.read_csv('./sample_without_index.csv')
df.head()

,Date,Price,Quantity,Width,Height,Quality,Score,Difference,Color,Shape
0,1997-07-05,2291,25,2.946650,5.305868,45.893300,52.762659,0.276266,green,triangle
1,1997-07-06,506,16,1.915208,0.679004,50.611735,31.453719,-1.854628,blue,NaN
2,1997-07-07,9629,32,7.869855,6.563335,43.830416,56.239011,0.623901,blue,square
3,1997-07-08,6161,67,6.375209,5.756029,41.358007,61.453113,1.145311,green,square
4,NaN,8570,55,0.390629,3.578136,55.739709,NaN,1.037190,red,square


In [27]:
# カテゴリカルな特徴慮color, shapeのみを扱う
df = df[['Color', 'Shape']]

In [28]:
df.head()

,Color,Shape
0,green,triangle
1,blue,NaN
2,blue,square
3,green,square
4,red,square


In [29]:
df.isnull().sum()

Color    4
Shape    5
dtype: int64

In [30]:
df[df['Color'].isnull()].index

Index([19, 37, 40, 73], dtype='int64')

## LabelEncoder
クラス数がn個あるとすると、それらを0,1,...,n-1にencodeする

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
encoder = LabelEncoder()

In [33]:
encoder.fit(df['Color'])

LabelEncoder()

In [34]:
encoder.classes_

array(['blue', 'green', 'red', nan], dtype=object)

In [35]:
encoder.transform(df['Color'])

array([1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 1, 0, 1, 1, 2, 0, 0, 0, 0, 3, 1, 0,
       1, 0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 2, 0, 1, 1, 3, 0, 0, 3, 2, 1, 2,
       0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2, 1,
       2, 1, 1, 2, 1, 0, 1, 3, 1, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0, 2, 0,
       1, 2, 2, 2, 0, 1, 2, 0, 2, 0, 0, 2])

In [36]:
df.head() # greenが1, blueが0, redが2になっていることがわかる(nanは3)

,Color,Shape
0,green,triangle
1,blue,NaN
2,blue,square
3,green,square
4,red,square


In [37]:
df_ce = df.copy()
df_ce['Color_encoded'] = encoder.transform(df['Color'])
df_ce = df_ce[['Color', 'Color_encoded', 'Shape']]
df_ce.loc[36:42]

,Color,Color_encoded,Shape
36,green,1,square
37,NaN,3,circle
38,blue,0,square
39,blue,0,square
40,NaN,3,triangle
41,red,2,NaN
42,green,1,square


In [38]:
# 逆変換
encoder.inverse_transform(df_ce.loc[36:42, 'Color_encoded'])

array(['green', nan, 'blue', 'blue', nan, 'red', 'green'], dtype=object)

## ダミー変数化

In [39]:
pd.get_dummies(df['Color']).head()

,blue,green,red
0,False,True,False
1,True,False,False
2,True,False,False
3,False,True,False
4,False,False,True


In [40]:
pd.get_dummies(df['Color'], drop_first=True).head()

,green,red
0,True,False
1,False,False
2,False,False
3,True,False
4,False,True


`get_dummies`で`drop_first=True`を指定するとこのようにblueが消える。
このとき、(True, False)の時はgreenを表していて(False, True)のときはredを表していて(False, False)のときはblueを表していることがわかる

In [41]:
df_cd = pd.get_dummies(df, columns=['Color'], drop_first=True)

In [42]:
df_cd.head()

,Shape,Color_green,Color_red
0,triangle,True,False
1,NaN,False,False
2,square,False,False
3,square,True,False
4,square,False,True


In [44]:
df_cd.isnull().sum() # 欠損の確認

Shape          5
Color_green    0
Color_red      0
dtype: int64

In [45]:
df_cd.loc[36:42]

,Shape,Color_green,Color_red
36,square,True,False
37,circle,False,False
38,square,False,False
39,square,False,False
40,triangle,False,False
41,NaN,False,True
42,square,True,False


このようにもともとnanが入っていた箇所には(False, False)が入ってblueと認識されるようになってしまっている。
そこで、`dummy_na=True`を追加することで欠損値を別の値として認識できる。
実際に下の例ではColor_nanという新しい特徴量が生成されていることがわかる。

In [46]:
df_cd = pd.get_dummies(df, columns=['Color'], drop_first=True, dummy_na=True)
df_cd.loc[36:42]

,Shape,Color_green,Color_red,Color_nan
36,square,True,False,False
37,circle,False,False,True
38,square,False,False,False
39,square,False,False,False
40,triangle,False,False,True
41,NaN,False,True,False
42,square,True,False,False


## OneHotEncoder

In [47]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder() # インスタンスの作成

In [49]:
encoder.fit(df[['Color']]) # LabelEncoderと異なり2次元で渡す必要がある

OneHotEncoder()

In [50]:
encoder.categories_

[array(['blue', 'green', 'red', nan], dtype=object)]

In [51]:
encoder.transform(df[['Color']]) # 2次元で渡す

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100 stored elements and shape (100, 4)>

In [53]:
encoder.transform(df[['Color']]).toarray()[:5]

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

ここからは複数の特徴量を一括で変更する

In [54]:
encoder = OneHotEncoder()

In [55]:
encoder.fit(df)

OneHotEncoder()

In [56]:
encoder.categories_

[array(['blue', 'green', 'red', nan], dtype=object),
 array(['circle', 'square', 'triangle', nan], dtype=object)]

In [57]:
encoder.transform(df).toarray()[:5] # これで一括で変換

array([[0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0.]])

blue, green, red, nan, circle, square, triangle, nanという並びになっている

In [58]:
# 逆変換
encoder.inverse_transform([[0, 1, 0, 0, 0, 1, 0, 0]])

array([['green', 'square']], dtype=object)

## `pd.get_dummies()`と`OneHotEncoder()`の違い

In [61]:
np.random.seed(1)
s = pd.Series(np.random.choice([0, 1], len(df)), name='target')
s

0     1
1     1
2     0
3     0
4     1
     ..
95    0
96    1
97    1
98    0
99    1
Name: target, Length: 100, dtype: int64

In [62]:
df_new = pd.concat([df, s], axis=1)
df_new.head()

,Color,Shape,target
0,green,triangle,1
1,blue,NaN,1
2,blue,square,0
3,green,square,0
4,red,square,1


In [63]:
from sklearn.model_selection import train_test_split

In [64]:
y = df_new.pop('target') # dfからtargetを切り取ってyに格納
X = df_new # popをすると消えるのでtarget以外がXに入る

In [66]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.05, stratify=y, random_state=17)

In [67]:
X_test

,Color,Shape
16,blue,circle
29,green,triangle
80,blue,triangle
44,blue,triangle
48,blue,triangle


In [68]:
pd.get_dummies(X_train, drop_first=True, dummy_na=True).head()

,Color_green,Color_red,Color_nan,Shape_square,Shape_triangle,Shape_nan
94,False,True,False,True,False,False
3,True,False,False,True,False,False
25,True,False,False,False,True,False
42,True,False,False,True,False,False
69,False,True,False,False,True,False


In [70]:
pd.get_dummies(X_test, drop_first=True, dummy_na=True).head()

,Color_green,Color_nan,Shape_triangle,Shape_nan
16,False,False,False,False
29,True,False,True,False
80,False,False,True,False
44,False,False,True,False
48,False,False,True,False


このようにget_dummiesを使用するとtrain, testでデータの偏りのせいで結果が変わってしまうかもしれない。

In [71]:
encoder = OneHotEncoder()

In [73]:
encoder.fit_transform(X_train).toarray()[:5]

array([[0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0.]])

In [74]:
encoder.transform(X_test).toarray()[:5] # trainをfitさせているためget_dummiesのように特徴量が減ることはない

array([[1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]])

以下、trainになくてtestに新しいラベルがあるような状況を考える。

このとき、OneHotEncoderのhandle_unknownを使う。

handle_unknownはerror, ignoreの2つのうちどちらかを指定できる

In [76]:
# handle_unknown='error'の場合
encoder_error = OneHotEncoder(handle_unknown='error', drop='first')

In [78]:
encoder_error.fit_transform(X_train).toarray()[:5] # blueとcircleを除いた6列

array([[0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.]])

In [79]:
encoder_error.transform(X_test).toarray()[:5]

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [80]:
X_test_new = X_test.copy()
X_test_new.loc[16, 'Color'] = 'purple' # 新しい値追加
encoder_error.transform(X_test_new)

ValueError: Found unknown categories ['purple'] in column 0 during transform

このようにValueErrorが発生するようになる

In [83]:
# ignoreの場合
encoder_ignore = OneHotEncoder(handle_unknown='ignore')

In [84]:
encoder_ignore.fit(X_train)

OneHotEncoder(handle_unknown='ignore')

In [86]:
encoder_ignore.transform(X_test).toarray()

array([[1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]])

In [87]:
encoder_ignore.transform(X_test_new).toarray()

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]])

このようにignoreではerrorが出ることなく、新しい値(purple)に対してすべて0になっていることがわかる。(1行目)

## Seriesのstr属性

In [88]:
df = pd.DataFrame()
df['ID'] = ['A-123', 'B-456', 'A-789', 'B-123']
df['Color'] = ['py/white black', 'red green blue', 'py/yellow', 'purple white']
df

,ID,Color
0,A-123,py/white black
1,B-456,red green blue
2,A-789,py/yellow
3,B-123,purple white


In [89]:
df['ID'].str # 指定したカラムの各値を文字列として抽出

In [93]:
df['ID'].str[0] # 各レコードの1文字目を取得

0    A
1    B
2    A
3    B
Name: ID, dtype: object

In [95]:
df['ID'].str[:] # 全部そのまま取得

0    A-123
1    B-456
2    A-789
3    B-123
Name: ID, dtype: object

In [96]:
df['ID'].str.lower() # 小文字に変換

0    a-123
1    b-456
2    a-789
3    b-123
Name: ID, dtype: object

In [97]:
df['ID'].str.startswith('A') # Aから始まるかどうか

0     True
1    False
2     True
3    False
Name: ID, dtype: bool

In [98]:
df['Color'].str.contains('white') # ある文字列を含むかどうか

0     True
1    False
2    False
3     True
Name: Color, dtype: bool

In [99]:
# yeを含む または puを含む
df['Color'].str.contains('ye|pu')

0    False
1    False
2     True
3     True
Name: Color, dtype: bool

In [100]:
df['Color'].str.replace('black', 'gold') # 置換

0     py/white gold
1    red green blue
2         py/yellow
3      purple white
Name: Color, dtype: object

In [101]:
df['ID'].str.split('-') # 分割

0    [A, 123]
1    [B, 456]
2    [A, 789]
3    [B, 123]
Name: ID, dtype: object

In [102]:
df['ID'].str.split('-', expand=True) # カラム追加

,0,1
0,A,123
1,B,456
2,A,789
3,B,123


In [104]:
df[['ID_a', 'ID_n']] = df['ID'].str.split('-', expand=True)
df

,ID,Color,ID_a,ID_n
0,A-123,py/white black,A,123
1,B-456,red green blue,B,456
2,A-789,py/yellow,A,789
3,B-123,purple white,B,123


In [105]:
df[['Color_1', 'Color_2', 'Color_3']] = df['Color'].str.split(' ', expand=True)
df

,ID,Color,ID_a,ID_n,Color_1,Color_2,Color_3
0,A-123,py/white black,A,123,py/white,black,None
1,B-456,red green blue,B,456,red,green,blue
2,A-789,py/yellow,A,789,py/yellow,None,None
3,B-123,purple white,B,123,purple,white,None


In [107]:
df['Color_1'].str.extract('(py/)', expand=True) # 抽出

,0
0,py/
1,NaN
2,py/
3,NaN


In [108]:
df['py'] = df['Color_1'].str.extract('(py/)', expand=True)
df

,ID,Color,ID_a,ID_n,Color_1,Color_2,Color_3,py
0,A-123,py/white black,A,123,py/white,black,None,py/
1,B-456,red green blue,B,456,red,green,blue,NaN
2,A-789,py/yellow,A,789,py/yellow,None,None,py/
3,B-123,purple white,B,123,purple,white,None,NaN


In [110]:
df['Color_1'] = df['Color_1'].str.replace('py/', '')
df

,ID,Color,ID_a,ID_n,Color_1,Color_2,Color_3,py
0,A-123,py/white black,A,123,white,black,None,py/
1,B-456,red green blue,B,456,red,green,blue,NaN
2,A-789,py/yellow,A,789,yellow,None,None,py/
3,B-123,purple white,B,123,purple,white,None,NaN
